In [1]:
from start_chrome_driver import start_google_chrome, REMOVE_ADS_SCRIPT
from selenium.webdriver.common.by import By
from time import sleep
import sqlite3

In [2]:
DATA = "ホールデータ"
URL = f"https://ana-slo.com/{DATA}"
PREF = "東京都"

DB_NAME = "../anaslo_02/anaslo_02.db"

In [3]:
driver = start_google_chrome("https://google.com")
driver.get(URL)
driver.execute_script(REMOVE_ADS_SCRIPT)
pref_ele = driver.find_elements(By.CLASS_NAME, "pref_list a")


--------------------------------------------------
実行成功: https://google.com


In [5]:
for pref in pref_ele:
    if pref.text == PREF:
        prefecture_name = pref.text
        print(f"都道府県: {prefecture_name}")
        break


都道府県: 東京都


In [6]:
pref.click()
driver.execute_script(REMOVE_ADS_SCRIPT)


In [8]:

# データベース登録
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

cursor.execute(
    "SELECT prefecture_id FROM prefectures WHERE name = ?", (prefecture_name,)
)
result = cursor.fetchone()
if result is None:
    print(f"❌ 都道府県が見つかりません: {prefecture_name}")

prefecture_id = result[0]
print(f"都道府県ID: {prefecture_id}")

hall_ele = driver.find_elements(By.CLASS_NAME, "table-body a")
for i, h in enumerate(hall_ele):
    cursor.execute(
        """
        INSERT OR IGNORE INTO halls (name, prefecture_id)
        VALUES (?, ?)
        """,
        (h.text.strip(), prefecture_id),
    )

print(f"ホール数: {len(hall_ele)}")
print("ホールデータを登録しました。")
        
conn.commit()
conn.close()

都道府県ID: 13
ホール数: 349
ホールデータを登録しました。
